<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Explore-other-reward-functions" data-toc-modified-id="Explore-other-reward-functions-1">Explore other reward functions</a></span><ul class="toc-item"><li><span><a href="#Goal" data-toc-modified-id="Goal-1.1">Goal</a></span></li><li><span><a href="#Imports" data-toc-modified-id="Imports-1.2">Imports</a></span></li><li><span><a href="#Ranking-reward" data-toc-modified-id="Ranking-reward-1.3">Ranking reward</a></span><ul class="toc-item"><li><span><a href="#Generate-games" data-toc-modified-id="Generate-games-1.3.1">Generate games</a></span><ul class="toc-item"><li><span><a href="#Code" data-toc-modified-id="Code-1.3.1.1">Code</a></span></li><li><span><a href="#Generation" data-toc-modified-id="Generation-1.3.1.2">Generation</a></span></li></ul></li></ul></li></ul></li></ul></div>

# Explore other reward functions

## Goal

The goal of this notebooks is to continue learning about the Q value function, but trying other reward methods.

## Imports

In [1]:
# Use this to reload changes in python scripts
%load_ext autoreload
%autoreload 2

In [2]:
import os
import kaggle_environments
from kaggle_environments import make
from kaggle_environments import evaluate
from kaggle_environments.envs.hungry_geese.hungry_geese import Action
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import random
import pandas as pd
from tqdm.notebook import tqdm
import time
from itertools import permutations
from functools import partial

os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'
os.environ['CUDA_VISIBLE_DEVICES'] = ''
#os.environ['CUDA_VISIBLE_DEVICES'] = '0'

import tensorflow as tf
import tensorflow.keras as keras

from hungry_geese import GameState
from hungry_geese.state import make_board_egocentric, get_head_position, combine_data, horizontal_simmetry, vertical_simmetry
from hungry_geese.utils import ACTIONS, opposite_action, get_timestamp
from hungry_geese.definitions import ACTION_TO_IDX
from hungry_geese.agents import EpsilonAgent
from hungry_geese.evaluation import play_matches_in_parallel, monitor_progress
from hungry_geese.elo import EloRanking
from hungry_geese.model import simple_model, create_model_for_training

%config Completer.use_jedi = False

Loading environment football failed: No module named 'gfootball'


In [3]:
plt.plot()
plt.close('all')
plt.rcParams["figure.figsize"] = (30, 5)  
mpl.rcParams['lines.linewidth'] = 3
mpl.rcParams['font.size'] = 16

## Ranking reward

### Generate games

#### Code

In [4]:
from concurrent.futures import ProcessPoolExecutor

def play_matches_in_parallel_and_save_history(agents, max_workers=20, n_matches=1000,
                                              running_on_notebook=True):
    """
    Plays n_matches in parallel using ProcessPoolExecutor

    Parameters
    -----------
    agents : dict
        Dictionary that matches name of the agents with the code
    sample_agents_func : func
        Function that returns random keys of the agents for playing a game
    """
    pool = ProcessPoolExecutor(max_workers=max_workers)
    matches_results = []
    submits = []
    for i in range(n_matches):
        submits.append(pool.submit(play_game, agents=agents))
    monitor_progress(submits, running_on_notebook)
    matches_results = [submit.result() for submit in submits]
    return matches_results

def play_game(agents):
    env = make("hungry_geese")
    return env.run(agents=agents)

In [5]:
def test_agent(agent):
    print('Testing agent: %s' % agent)
    env = make('hungry_geese', debug=True)
    env.run([agent]*4)

In [6]:
def create_train_data(agents, n_matches, output_file, reward_name, max_workers=18):
    """
    Creates train data without any simmetry
    """
    test_agent(agents[0])
    matches_results = play_matches_in_parallel_and_save_history(
        agents, n_matches=n_matches, max_workers=max_workers)
    
    env = make("hungry_geese")
    conf = env.configuration

    state = GameState(reward_name=reward_name)
    train_data = []

    for match in tqdm(matches_results):
        for idx in range(4):
            state.reset()
            for step_idx, step in enumerate(match):
                observation = step[0]['observation'].copy()
                observation['index'] = idx
                state.update(observation, conf)
                if step_idx:
                    state.add_action(step[idx]['action'])
                if not observation['geese'][idx]:
                    break
            train_data.append(state.prepare_data_for_training())

    train_data = combine_data(train_data)
    print('Size of the boards is %.1f GB' % (train_data[0].nbytes/1e9))
    os.makedirs(os.path.dirname(output_file), exist_ok=True)
    np.savez_compressed(output_file, boards=train_data[0], features=train_data[1], actions=train_data[2], rewards=train_data[3])

#### Generation

In [7]:
n_matches = 10
agents = ['/mnt/hdd0/MEGA/AI/22 Kaggle/hungry_geese/forum/agents/random_plus.py']*4
reward_name = 'ranking_reward_-5_2'
output_file = '/mnt/hdd0/Kaggle/hungry_geese/data/%s/%s_matches_%i.npz' % (reward_name, os.path.splitext(os.path.basename(agents[0]))[0], n_matches)
create_train_data(agents=agents, n_matches=n_matches, output_file=output_file, reward_name=reward_name, max_workers=18)

Testing agent: /mnt/hdd0/MEGA/AI/22 Kaggle/hungry_geese/forum/agents/random_plus.py
Goose Collision: EAST
Goose Collision: NORTH
Goose Starved: Action.SOUTH
Goose Starved: Action.WEST


  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

TypeError: No loop matching the specified signature and casting was found for ufunc true_divide